In [1]:
from dotenv import load_dotenv
load_dotenv()

True

```python
LANGCHAIN_TRACING_V2=True  # 켜면 모든 실행 기록을 LangSmith에 보냄
LANGCHAIN_TRACING_V2=False # 끄면 추적 안 함 (에러 없음)
```

In [2]:
from langchain_teddynote import logging
logging.langsmith("Cache_FAISS")

LangSmith 추적을 시작합니다.
[프로젝트명]
Cache_FAISS


In [3]:
from langchain_community.document_loaders import TextLoader

# Markdown 파일 경로
FILE_PATH = "../ParsingData/LlamaParse_Multimodal(gpt-4o-mini).md"

# TextLoader로 불러오기
loader = TextLoader(FILE_PATH, encoding="utf-8")
docs = loader.load()
len(docs)

1

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

documents = text_splitter.split_documents(docs)
len(documents)

37

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# 벡터 저장소로 변환
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(documents, embedding)

retriever = vectorstore.as_retriever()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17228\3541954798.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model="text-embedding-3-small")


### 임베딩 & 벡터 DB 에 저장

캐시형 임베딩을 사용해, 이전에 계산된 임베딩을 캐시에서 불러오는 방식

In [6]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 로컬파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

db = FAISS.from_documents(documents, cached_embedder)

In [7]:
FAISS_DB_PATH = "cache"
db.save_local(FAISS_DB_PATH)

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# allow_dangerous_deserialization: Pickle을 통한 위험한 객체 로드를 허용. 악성 코드가 실행될 가능성 있음
vectorstore = FAISS.load_local(
    FAISS_DB_PATH, embeddings, allow_dangerous_deserialization=True
)

In [9]:
retriever = vectorstore.as_retriever()

In [10]:
retriever.invoke("삼성전자 생성형 ai의름은?")

[Document(id='9e19f8fb-badb-46e0-93f9-f3cccc3ff445', metadata={'source': '../ParsingData/LlamaParse_Multimodal(gpt-4o-mini).md'}, page_content='삼성 가우스는 스텍스트를 생성하는 언어모델 스코드를 생성하는 코드 모델 스미지를 생성하는 이미지 모델의 3개 모델로 구성\n\n| 모델     | 설명                                                                |\n| ------ | ----------------------------------------------------------------- |\n| 언어 모델  | 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 이메일 작성, 문서 요약, 번역 업무 처리 지원           |\n| 코드 모델  | AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 사내 소프트웨어 개발에 최적화 |\n| 이미지 모델 | 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 저해상도 이미지의 고해상도 전환도 지원 |\n\nIT 전문 매체 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했으며, 2024년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)를 탑재한 웨얼러 기기 구글 어시스턴트를 적용한 구글 픽셀(Pixel)과 경쟁할 것으로 예상\n\n출처: 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성 AI ‘삼성 가우스’ 공개, 2023.11.08\n\n삼성전자, ‘삼성 개발자 컨퍼런스 코리아 2023’ 개최, 2023.11.14.\n\nTechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.\n\n# 구글, 앤스프릭에 최대 20억 달러 투자로 생성 AI 협력 강화